In [1]:
from model import FCAE
from dataGenTab import DataGen
from utils import utils
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
import torch


/home/savvas/anaconda3/envs/PyTorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Datafeeder

In [2]:
config_path = 'config.json'
config = utils.Configuration(config_path)

df = pd.read_csv(os.path.join(config.datapath,'data_aet.csv'))
ids = list(df['cell-id'])
ids = sorted(ids)
print(len(ids))


train_dataset = DataGen(ids,  config_path=config_path)


train_loader = DataLoader(train_dataset, batch_size=config.batch_size)


4864


## Model

In [3]:
input_dim = 14 * 12  # Flattened input
filters = [128,64,32] #last number for latentspace
model = FCAE(input_dim, filters)
state_dict = torch.load('./train-results/weathermodel-3/weathermodel-3_final.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()



FCAE(
  (encoder): Sequential(
    (0): Linear(in_features=168, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=168, bias=True)
    (5): Sigmoid()
  )
)

In [10]:
embedding_df = pd.DataFrame()
with torch.no_grad():
    for i, data in enumerate(train_loader):
        inputs, label, id = data
        outputs, embedding = model(inputs)
        embedding_np = embedding.cpu().numpy()
        
        # Create a temporary DataFrame
        temp_df = pd.DataFrame({
            'cell-id': int(id[0].numpy()),
            'embedding': [list(embedding_np[0])]
        })
        
        # Append to the main DataFrame
        embedding_df = pd.concat([embedding_df, temp_df], ignore_index=True)

        # print(id)

        # if i==2:
        #     break

embedding_df.to_csv('weather-embedding.csv', index=False)
embedding_df

,cell-id,embedding
0,1,"[0.0, 2.9571898, 0.0, 3.1007698, 0.0, 0.0, 2.1..."
1,2,"[0.0, 2.0444605, 0.0, 1.8483584, 0.0, 0.0, 2.4..."
2,3,"[0.0, 3.8294566, 0.0, 2.1116407, 0.0, 0.0, 1.6..."
3,4,"[0.0, 1.901012, 0.0, 2.4555795, 0.0, 0.0, 0.81..."
4,5,"[0.0, 2.6817884, 0.0, 1.9616616, 0.0, 0.0, 1.3..."
...,...,...
4859,4885,"[0.0, 0.0, 0.0, 2.2069588, 0.0, 0.0, 0.0, 0.0,..."
4860,4886,"[0.0, 0.0, 0.0, 2.1894383, 0.0, 0.0, 0.0, 0.0,..."
4861,4887,"[0.0, 0.0, 0.0, 2.0301235, 0.0, 0.0, 0.0, 0.0,..."
4862,4888,"[0.0, 0.0, 0.0, 2.063947, 0.0, 0.0, 0.0, 0.0, ..."
